In [ ]:
!pip install flashrank

In [ ]:
!pip install langchain langchain-community langchain-huggingface

In [ ]:
!pip install pypdf tiktoken

In [ ]:
%pip install chromadb

In [ ]:
%pip install rank_bm25

## STEP 1 (DATA INGESTION)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Docs.pdf to Docs.pdf


In [7]:
filePath = "/content/Docs.pdf"

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
def loadDocs(path:str):
  loader = PyPDFLoader(path)
  docs = loader.load()
  return docs

In [10]:
docs = loadDocs(filePath)
print(len(docs))
print(docs[0])

19
page_content='Classification of 
Parallel Computers 
 
 
                                                                                                                            
UNIT 2  CLASSIFICATION OF PARALLEL    
                COMPUTERS  
Structure          Page Nos. 
 
2.0 Introduction       27 
2.1   Objectives       27 
2.2   Types of Classification      28 
2.3  Flynn’s Classification      28 
2.3.1  Instruction Cycle 
2.3.2  Instruction Stream and Data Stream 
2.3.3  Flynn’s Classification 
2.4   Handler’s Classification      33 
2.5   Structural Classification      34 
2.5.1  Shared Memory System/Tightly Coupled System 
    2.5.1.1 Uniform Memory Access Model  
    2.5.1.2 Non-Uniform Memory Access Model  
           2.5.1.3 Cache-only Memory Architecture Model 
2.5.2  Loosely Coupled Systems 
2.6  Classification Based on Grain Size    39 
2.6.1 Parallelism Conditions 
2.6.2 Bernstein Conditions for Detection of Parallelism 
2.6.3 Parallelism Based on Grain Size 
2.7

In [11]:
def splitDocs(docs):
  splitter = RecursiveCharacterTextSplitter(
      chunk_size = 400,
      chunk_overlap = 150
  )
  chunks = splitter.split_documents(docs)
  return chunks

In [12]:
chunks = splitDocs(docs)
print(len(chunks))
print(chunks[0])

159
page_content='Classification of 
Parallel Computers 
 
 
                                                                                                                            
UNIT 2  CLASSIFICATION OF PARALLEL    
                COMPUTERS  
Structure          Page Nos. 
 
2.0 Introduction       27 
2.1   Objectives       27 
2.2   Types of Classification      28 
2.3  Flynn’s Classification      28' metadata={'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'creator': 'Acrobat PDFMaker 5.0 for Word', 'creationdate': '2007-04-09T09:34:41+05:30', 'author': 'Naresh Chauhan', 'moddate': '2007-04-09T09:35:40+05:30', 'title': 'UNIT 2 CLASSIFICATION OF PARALLEL', 'source': '/content/Docs.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint

In [116]:
# HF API

In [ ]:
embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFacePipeline
llm = HuggingFaceEndpoint(
    repo_id = "HuggingFaceH4/zephyr-7b-beta",
    task = "text-generation",
)
model = ChatHuggingFace(llm = llm)

In [19]:
parser = StrOutputParser()

## STEP 2 (HYBRID SEARCH)

In [20]:
from langchain_community.vectorstores import Chroma
vectorStore = Chroma.from_documents(chunks, embedding)

In [21]:
similarityRetirever = vectorStore.as_retriever(search_type = "similarity", search_kwargs = {"k":20})

In [22]:
from langchain_community.retrievers import BM25Retriever
keywordRetriever = BM25Retriever.from_documents(chunks)
keywordRetriever.k = 20

In [106]:
from langchain_community.vectorstores import Chroma
vectorStore = Chroma.from_documents(chunks, embedding)

similarityRetriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 20})

from langchain_community.retrievers import BM25Retriever
keywordRetriever = BM25Retriever.from_documents(chunks)
keywordRetriever.k = 20

def hybridRetriever(
    query,
    denseRetriever,
    sparseRetriever,
    denseWeight=0.5,
    sparseWeight=0.5,
    rrf_k=60
):
    scores = {}
    doc_map = {}

    # Dense retrieval
    denseDocs = denseRetriever.invoke(query)
    for rank, doc in enumerate(denseDocs):
        key = doc.page_content
        doc_map[key] = doc
        scores[key] = scores.get(key, 0) + denseWeight / (rank + 1 + rrf_k)

    # Sparse retrieval
    sparse_docs = sparseRetriever.invoke(query)
    for rank, doc in enumerate(sparse_docs):
        key = doc.page_content
        doc_map[key] = doc
        scores[key] = scores.get(key, 0) + sparseWeight / (rank + 1 + rrf_k)

    # Sort by final score
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    return [content for content, _ in ranked_docs]

In [107]:
query = "FLYNN’S CLASSIFICATION?"

results = hybridRetriever(
    query=query,
    denseRetriever= similarityRetirever,
    sparseRetriever=keywordRetriever,
    denseWeight=0.5,
    sparseWeight=0.5
)

print(len(results))
for i, r in enumerate(results):
    print(f"{i+1}. {r[:100]}")


28
1. 3) Classification based on how the memory is accessed 
4) Classification based on grain size 
 
All 
2. …………………………………………………………………………………………
…………………………………………………………………………………………
………………………………………………………………………………
3. form the Data streams. 
 
2.3.3 Flynn’s Classification 
Flynn’s classification is based on multiplic
4. not consider the machine architecture for classification of parallel computers; he 
introduced the c
5. parallel computers, it is necessary to understand all types of Flynn’s classification. Since, 
this 
6. Elements of Parallel 
Computing and  
Architecture 
 
 
Thus, it can be said that the sequence of in
7. 2.0 Introduction       27 
2.1   Objectives       27 
2.2   Types of Classification      28 
2.3  Fl
8. operator is of a different form to the other two in that it is used to combine the different 
operat
9. parallel computers based on the above mentioned criteria.  
 
2.1  OBJECTIVES 
After going through t
10. single processor, the variety of parallelism in multiproc

## STEP 3 (FLASH RERANKING)


In [ ]:
!pip install flashrank

In [50]:
from flashrank.Ranker import Ranker, RerankRequest

In [114]:
def reranking(query,passages,choice):
  if choice == "Nano":
    ranker = Ranker()
  elif choice == "Small":
    ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2", cache_dir="/opt")
  elif choice == "Medium":
    ranker = Ranker(model_name="rank-T5-flan", cache_dir="/opt")
  elif choice == "Large":
    ranker = Ranker(model_name="ms-marco-MultiBERT-L-12", cache_dir="/opt")
  rerankrequest = RerankRequest(query=query, passages=passages)
  results = ranker.rerank(rerankrequest)

  return results

In [110]:
hybridResults = hybridRetriever(
    query=query,
    denseRetriever=similarityRetriever,
    sparseRetriever=keywordRetriever
)

passages = [
    {"id": i, "text": str(chunk)}
    for i, chunk in enumerate(hybridResults)
]
print(passages[0])

{'id': 0, 'text': '3) Classification based on how the memory is accessed \n4) Classification based on grain size \n \nAll these classification schemes are discussed in subsequent sections. \n \n2.3  FLYNN’S CLASSIFICATION \nThis classification was first studied and proposed by Michael Flynn in 1972. Flynn did \nnot consider the machine architecture for classification of parallel computers; he'}


In [109]:
rerankingResults = reranking(query, passages, "Medium")
rerankingResults[0]

[{'id': 3, 'text': 'not consider the machine architecture for classification of parallel computers; he \nintroduced the concept of instruction and data streams for categorizing of computers. All \nthe computers classified by Flynn are not parallel computers, but to grasp the concept of \nparallel computers, it is necessary to understand all types of Flynn’s classification. Since,', 'score': np.float32(0.58191013)}, {'id': 0, 'text': '3) Classification based on how the memory is accessed \n4) Classification based on grain size \n \nAll these classification schemes are discussed in subsequent sections. \n \n2.3  FLYNN’S CLASSIFICATION \nThis classification was first studied and proposed by Michael Flynn in 1972. Flynn did \nnot consider the machine architecture for classification of parallel computers; he', 'score': np.float32(0.5497802)}, {'id': 5, 'text': 'Elements of Parallel \nComputing and  \nArchitecture \n \n \nThus, it can be said that the sequence of instructions executed by CPU

{'id': 3,
 'text': 'not consider the machine architecture for classification of parallel computers; he \nintroduced the concept of instruction and data streams for categorizing of computers. All \nthe computers classified by Flynn are not parallel computers, but to grasp the concept of \nparallel computers, it is necessary to understand all types of Flynn’s classification. Since,',
 'score': np.float32(0.58191013)}

## STEP 3 (CROSS-ENCODERS) RERANKING AGAIN FOR MORE ACCURATE RESULTS

In [62]:
from sentence_transformers import CrossEncoder
crossEncoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [70]:
topk = [item["text"] for item in rerankingResults]
topk[0]

'Elements of Parallel \nComputing and  \nArchitecture \n \n \nThus, it can be said that the sequence of instructions executed by CPU forms the \nInstruction streams and sequence of data (operands) required for execution of instructions \nform the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams'

In [75]:
pairs = [[query, text] for text in topk]
pairs[0]

['FLYNN’S CLASSIFICATION?',
 'Elements of Parallel \nComputing and  \nArchitecture \n \n \nThus, it can be said that the sequence of instructions executed by CPU forms the \nInstruction streams and sequence of data (operands) required for execution of instructions \nform the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams']

In [76]:
from sentence_transformers import CrossEncoder
crossEncoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
crossEncoderScores = crossEncoder.predict(pairs)

In [79]:
crossRerankedResults = list(zip(crossEncoderScores, topk))
crossRerankedResults = sorted(crossRerankedResults, reverse=True)
crossRerankedResults[0]

(np.float32(8.123538),
 'form the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams \nobserved by the CPU during program execution. Let Is and Ds  are minimum number of \nstreams flowing at any point in the execution, then the computer organisation can be \ncategorized as follows: \n1) Single Instruction and Single Data stream (SISD)')

In [82]:
rerankingResults[0]

{'id': 9,
 'text': 'Elements of Parallel \nComputing and  \nArchitecture \n \n \nThus, it can be said that the sequence of instructions executed by CPU forms the \nInstruction streams and sequence of data (operands) required for execution of instructions \nform the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams',
 'score': np.float32(0.5631269)}

In [85]:
crossEncoderK = 4
crossEncoderTopKFinalResults = [text for _, text in crossRerankedResults[:crossEncoderK]]
crossEncoderTopKFinalResults[0]

'form the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams \nobserved by the CPU during program execution. Let Is and Ds  are minimum number of \nstreams flowing at any point in the execution, then the computer organisation can be \ncategorized as follows: \n1) Single Instruction and Single Data stream (SISD)'

### STEP 4 (LongContextReorder) For Solving Lost in Middle Phenomenon

In [95]:
from langchain_core.documents import Document
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()

finalDocs = [
    Document(page_content=text)
    for text in crossEncoderTopKFinalResults
]

finalDocsReordered = reorder.transform_documents(finalDocs)

for i in range(len(finalDocsReordered)):
    print(f"({i+1}) --> {finalDocsReordered[i].page_content}")

(1) --> 3) Classification based on how the memory is accessed 
4) Classification based on grain size 
 
All these classification schemes are discussed in subsequent sections. 
 
2.3  FLYNN’S CLASSIFICATION 
This classification was first studied and proposed by Michael Flynn in 1972. Flynn did 
not consider the machine architecture for classification of parallel computers; he
(2) --> Elements of Parallel 
Computing and  
Architecture 
 
 
Thus, it can be said that the sequence of instructions executed by CPU forms the 
Instruction streams and sequence of data (operands) required for execution of instructions 
form the Data streams. 
 
2.3.3 Flynn’s Classification 
Flynn’s classification is based on multiplicity of instruction streams and data streams
(3) --> 2.0 Introduction       27 
2.1   Objectives       27 
2.2   Types of Classification      28 
2.3  Flynn’s Classification      28 
2.3.1  Instruction Cycle 
2.3.2  Instruction Stream and Data Stream 
2.3.3  Flynn’s Classification 
2.

### STEP 5 (AUGMENTATION)

In [96]:
def buildContext(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)
context = buildContext(finalDocsReordered)
context

'3) Classification based on how the memory is accessed \n4) Classification based on grain size \n \nAll these classification schemes are discussed in subsequent sections. \n \n2.3  FLYNN’S CLASSIFICATION \nThis classification was first studied and proposed by Michael Flynn in 1972. Flynn did \nnot consider the machine architecture for classification of parallel computers; he\n\n---\n\nElements of Parallel \nComputing and  \nArchitecture \n \n \nThus, it can be said that the sequence of instructions executed by CPU forms the \nInstruction streams and sequence of data (operands) required for execution of instructions \nform the Data streams. \n \n2.3.3 Flynn’s Classification \nFlynn’s classification is based on multiplicity of instruction streams and data streams\n\n---\n\n2.0 Introduction       27 \n2.1   Objectives       27 \n2.2   Types of Classification      28 \n2.3  Flynn’s Classification      28 \n2.3.1  Instruction Cycle \n2.3.2  Instruction Stream and Data Stream \n2.3.3  Flynn’

In [111]:
def generateResponse(question):
    """
    Complete RAG pipeline with hybrid retrieval, flash reranking,
    cross-encoder reranking, and long context reordering.
    """
    # Step 1: Hybrid Retrieval
    hybridResults = hybridRetriever(
        query=question,
        denseRetriever=similarityRetriever,
        sparseRetriever=keywordRetriever
    )

    # Step 2: Flash Reranking
    passages = [
        {"id": i, "text": str(chunk)}
        for i, chunk in enumerate(hybridResults)
    ]

    rerankingResults = reranking(question, passages, "Medium")
    flashTopK = rerankingResults[:8]

    # Step 3: Cross-Encoder Reranking
    topk_texts = [item["text"] for item in flashTopK]
    pairs = [[question, text] for text in topk_texts]

    crossEncoderScores = crossEncoder.predict(pairs)
    crossReranked = sorted(zip(crossEncoderScores, topk_texts), reverse=True)

    FINAL_K = 4
    finalTexts = [text for _, text in crossReranked[:FINAL_K]]

    # Step 4: Long Context Reorder
    finalDocs = [Document(page_content=text) for text in finalTexts]
    finalDocsReordered = reorder.transform_documents(finalDocs)

    # Step 5: Build Context
    context = buildContext(finalDocsReordered)

    # Step 6: Generation
    prompt = PromptTemplate(
        template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.

<context>
{context}
</context>

Question: {question}
Answer:
""",
        input_variables=["context", "question"]
    )

    chain = prompt | model | parser
    return chain.invoke({"context": context, "question": question})

In [115]:
while True:
    question = input("Enter your question related to the document: ")

    if not question.strip():
        print("Please enter a valid question.\n")
        continue

    print("\nGenerating answer...\n")
    answer = generateResponse(question)
    print("Answer:", answer)

    continue_chat = input("Do you want to ask another question? (yes/no): ").strip().lower()
    if continue_chat not in ['yes', 'y']:
        print("\nThank you for using the document Q&A system. Goodbye!")
        break

Enter your question related to the document: Explain loosely coupled systems

Generating answer...

Answer: In loosely coupled systems, also known as distributed multicomputer systems, processes communicate through message passing as each processor has its own local memory that is not accessible by other processors. This organization, where separate computers or nodes communicate through a message passing interconnection network, is called a distributed memory computer. These systems have less coupling between nodes, as shown in Figure 14. This type of architecture is distinguished from shared memory computers, where all processors have access to a common memory, also called tightly coupled systems. In loosely coupled systems, processors have their own local memory and message passing is necessary for inter-node communication due to the isolation of local memories. This organization involves multiple processors with their own local memory and I/O devices, forming a computer system with